In [4]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException ,StaleElementReferenceException
from selenium.webdriver.support import expected_conditions
#time
import time
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests_html import HTMLSession
from unidecode import unidecode
from datetime import date
from bs4 import BeautifulSoup
import requests


In [22]:
def full():

    PATH= '/usr/lib/chromium-browser/chromedriver'

    link = 'https://diaonline.supermercadosdia.com.ar/'

    s= Service(PATH)

    driver = webdriver.Chrome(service=s)

    driver.set_window_size(1366,768)

    driver.get(link)

    time.sleep(12)

    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)

    buttons_a= driver.find_elements(By.CSS_SELECTOR,'div.vtex-menu-2-x-styledLinkContainer.vtex-menu-2-x-styledLinkContainer--compra-online')


    categoria=[]
    for i in buttons_a:
        #print(i.text)
        categoria.append(unidecode(i.text).lower()) #saco tildes y hago minuscula


    buttons_b= driver.find_elements(By.CSS_SELECTOR,'div.vtex-menu-2-x-styledLinkContainer.vtex-menu-2-x-styledLinkContainer--compra-online-2')

    for i in buttons_b:
        
        categoria.append(unidecode(i.text).lower()) #sacar tildes y hacerlo minuscula

    full_link=[]
    for elemento in categoria:
        if " " in elemento:
            sub=elemento.split(" ")

            substring="-".join(sub)
            if substring == 'frutas-y-verduras':
                corregido="/".join([ 'frescos','frutas-y-verduras'])
                full_link.append(link+corregido) #sumo link y la parte de la categoria
            else:

                full_link.append(link+substring) #sumo link y la parte de la categoria

        else:
            full_link.append(link+elemento)

    df = pd.DataFrame({'categoria':categoria,'links': full_link})

    return df

data=full()


today = date.today() #hold the date
d1 = today.strftime("%Y%m%d") 
data.to_csv(f'../csv/dia/{d1}dia_links.csv', encoding='utf-8', index=False) #always printing the date of the csv file


KeyboardInterrupt: 

In [56]:
df = pd.read_csv('../csv/dia/20230411dia_links.csv')

df

,categoria,links
0,especial ofertas,https://diaonline.supermercadosdia.com.ar/espe...
1,almacen,https://diaonline.supermercadosdia.com.ar/almacen
2,desayuno,https://diaonline.supermercadosdia.com.ar/desa...
3,bebidas,https://diaonline.supermercadosdia.com.ar/bebidas
4,frescos,https://diaonline.supermercadosdia.com.ar/frescos
5,frutas y verduras,https://diaonline.supermercadosdia.com.ar/fres...
6,congelados,https://diaonline.supermercadosdia.com.ar/cong...
7,perfumeria,https://diaonline.supermercadosdia.com.ar/perf...
8,limpieza,https://diaonline.supermercadosdia.com.ar/limp...
9,bebes y ninos,https://diaonline.supermercadosdia.com.ar/bebe...


In [6]:
def products(categoria,source):

    

    #r = requests.get(url)


    soup =BeautifulSoup(source,'html.parser')

    #frame = soup.find('div',class_='vtex-search-result-3-x-gallery vtex-search-result-3-x-gallery--default flex flex-row flex-wrap items-stretch bn ph1 na4 pl9-l')

    #print(frame)
    
    products =soup.find_all('div',class_='vtex-search-result-3-x-galleryItem vtex-search-result-3-x-galleryItem--normal vtex-search-result-3-x-galleryItem--default pa4')
    
    count_product=0
    product_list=[]
    for product in products:
         
        count_product=count_product+1
    #this is the count fraction to check the scrapped elements
        print(f'{count_product} / {len(products)} Elemento escrapeado')

        current_product={'market':'Dia','category': categoria,'currency':'ARG'}

        #marca producto
        brand= product.find('div',class_='vtex-product-summary-2-x-productBrandContainer').text
        current_product['marca']=brand

        #precio descripcion
        description= product.find('div',class_='vtex-product-summary-2-x-nameContainer flex items-start justify-center pv6').text
        current_product['descripcion'] =description

        #promocion
        try:
            promocion= product.find('div',class_='vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--shelf_promotions').text
            current_product['promocion'] = promocion
        except:
            current_product['promocion'] = None

        #precio producto
        precio = product.find('div',class_='vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--shelf_prices').text
    
        if precio.count('$',2):
            part = precio.split("$")          
            precio= "$"+part[1]
            precio_promocion="$"+part[2]
        else:
            precio
            
            precio_promocion= None
        
    
        
        current_product['precio_producto'] =precio
        current_product['precio_promocion'] = precio_promocion

        #precio x unidad
        precio_unidad = product.find('div',class_='vtex-flex-layout-0-x-flexRow vtex-flex-layout-0-x-flexRow--product-unit').text
        current_product['precio_kilo']= precio_unidad
        

        product_list.append(current_product)


    return product_list





In [12]:

def full_scrap(categoria,link):


    PATH= '/usr/lib/chromium-browser/chromedriver'

    

    s= Service(PATH)

    driver = webdriver.Chrome(service=s)

    driver.maximize_window()

    driver.get(link)

    time.sleep(12)


    cont = 0
    prev_url = None  # Almacenar el valor anterior de driver.current_url

    while True:
    
        if driver.current_url == prev_url:
        # Salir del bucle si driver.current_url no ha cambiado
            break
    
        prev_url = driver.current_url  # Actualizar el valor anterior de driver.current_url
    
        cont=cont+1
        print(f'cantidad de hojas: {cont}')
        elemento = driver.find_element(By.CSS_SELECTOR, 'div.vtex-search-result-3-x-buttonShowMore--layout')
        driver.execute_script("arguments[0].scrollIntoView(0);", elemento)
        time.sleep(10)
    

    
    data=products(categoria,driver.page_source)

    df=pd.DataFrame(data).replace("",None)
       

    driver.quit()

    return df

In [5]:
main_data= pd.read_csv('../csv/dia/20230411dia_links.csv')

In [10]:
main_data= main_data.loc[1:]

In [13]:
main=[full_scrap(i,u) for i,u in zip(main_data['categoria'],main_data['links'])]

cantidad de hojas: 1
cantidad de hojas: 2
cantidad de hojas: 3
cantidad de hojas: 4
cantidad de hojas: 5
cantidad de hojas: 6
cantidad de hojas: 7
cantidad de hojas: 8
cantidad de hojas: 9
cantidad de hojas: 10
cantidad de hojas: 11
cantidad de hojas: 12
cantidad de hojas: 13
cantidad de hojas: 14
cantidad de hojas: 15
cantidad de hojas: 16
cantidad de hojas: 17
cantidad de hojas: 18
cantidad de hojas: 19
cantidad de hojas: 20
cantidad de hojas: 21
cantidad de hojas: 22
cantidad de hojas: 23
cantidad de hojas: 24
cantidad de hojas: 25
cantidad de hojas: 26
cantidad de hojas: 27
cantidad de hojas: 28
cantidad de hojas: 29
cantidad de hojas: 30
cantidad de hojas: 31
cantidad de hojas: 32
cantidad de hojas: 33
cantidad de hojas: 34
cantidad de hojas: 35
cantidad de hojas: 36
cantidad de hojas: 37
cantidad de hojas: 38
cantidad de hojas: 39
cantidad de hojas: 40
cantidad de hojas: 41
cantidad de hojas: 42
cantidad de hojas: 43
cantidad de hojas: 44
cantidad de hojas: 45
cantidad de hojas: 

In [14]:
main

[    market category currency      marca  \
 0      Dia  almacen      ARG  SIN MARCA   
 1      Dia  almacen      ARG  SIN MARCA   
 2      Dia  almacen      ARG   LEGITIMO   
 3      Dia  almacen      ARG      ARCOR   
 4      Dia  almacen      ARG    FERRERO   
 ..     ...      ...      ...        ...   
 795    Dia  almacen      ARG    LULEMUU   
 796    Dia  almacen      ARG    LULEMUU   
 797    Dia  almacen      ARG    LULEMUU   
 798    Dia  almacen      ARG       None   
 799    Dia  almacen      ARG     KANSAS   
 
                                            descripcion promocion  \
 0                 Harina De Trigo 0000 Cañuelas 1 Kg.       None   
 1                  Aceite de Girasol Legítimo 900 Ml.       None   
 2                  Aceite de Girasol Legítimo 1,5 Lt.       None   
 3                        Puré de Tomate Arcor 530 Gr.       None   
 4    Huevo de Pascua Ferrero Rocher con Avellanas 2...       50%   
 ..                                                 ... 

In [21]:
dfs=[pd.DataFrame(i) for i in main]
df=pd.concat(dfs)
df

,market,category,currency,marca,descripcion,promocion,precio_producto,precio_promocion,precio_kilo
0,Dia,almacen,ARG,SIN MARCA,Harina De Trigo 0000 Cañuelas 1 Kg.,None,"$ 144,27",None,0.14GR
1,Dia,almacen,ARG,SIN MARCA,Aceite de Girasol Legítimo 900 Ml.,None,"$ 279,50",None,310.56LT
2,Dia,almacen,ARG,LEGITIMO,"Aceite de Girasol Legítimo 1,5 Lt.",None,"$ 446,50",None,297.67LT
3,Dia,almacen,ARG,ARCOR,Puré de Tomate Arcor 530 Gr.,None,"$ 167,70",None,316.42KG
4,Dia,almacen,ARG,FERRERO,Huevo de Pascua Ferrero Rocher con Avellanas 2...,50%,"$ 1.824,50","$ 3.649,00",8108.89KG
...,...,...,...,...,...,...,...,...,...
152,Dia,electro hogar,ARG,ELECTROLUX,Secarropas Electrolux Eseb6 6kg,8%,"$ 141.399,00","$ 153.299,00",None
153,Dia,electro hogar,ARG,WHIRLPOOL,Accesorio KitchenAid Gancho amasador de aluminio,5%,"$ 11.199,00","$ 11.799,00",None
154,Dia,electro hogar,ARG,WHIRLPOOL,Accesorio KitchenAid Batidor plano Artisan,5%,"$ 6.599,00","$ 6.969,00",None
155,Dia,electro hogar,ARG,WHIRLPOOL,Accesorio Escudo vertedor Heavy Duty,5%,"$ 8.999,00","$ 9.499,00",None
